In [132]:
import pandas as pd
import requests

In [204]:
data = pd.read_csv("./products.csv", sep=";", encoding='utf-8')
currency_country = pd.read_json("currency.json", typ="series")
euro_rate = pd.read_json("eur.json")
data.head(10)

,username,ip_address,product,price,infos
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish"
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33 MGA,Ingredients: sugar and milk
6,afairholme6,127.197.204.119,Chicken - Tenderloin,484.83,May contain sugar
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar
8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish"
9,tkendrew9,22.32.234.215,Pop - Club Soda Can,861.25,"May contain peanut, sugar, milk and fish"


In [205]:
#Check for currency after the price
def has_currency(price):
    priceSplit = price.split(" ")
    if(len(priceSplit) == 2):
        return priceSplit[1]
    else:
        return None

data["currency"] = data["price"].apply(has_currency)
data["price"] = data["price"].apply(lambda price: price.split(" ")[0])
data["price_eur"] = None
data = data.astype({'price_eur': np.float, 'price': np.float})

In [189]:
data

,username,ip_address,product,price,infos,currency,price_eur
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar,None,None
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,None,None
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,None,None
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,None,None
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",None,None
...,...,...,...,...,...,...,...
195,ccannop5f,174.5.73.129,Nantucket Orange Juice,713.6,"Contains gluten, mustard, sugar, milk and fish",None,None
196,lprovis5g,191.69.45.257,"Jam - Strawberry, 20 Ml Jar",107.78,Contains gluten and sugar,None,None
197,iollarenshaw5h,206.30.25.226,"Juice - Apple, 341 Ml",447.17,May contain gluten and soja,RUB,None
198,mjablonski5i,44.133.211.182,"Mushroom - Chantrelle, Fresh",632.29,Ingredients: soja and sugar,None,None


In [206]:
def get_eur_price(row, currency_country, euro_rate):
    locationIp = requests.get("https://freegeoip.app/json/" + row["ip_address"])
    if (locationIp.status_code == 200):
        locationIp = locationIp.json()
    else:
        return row
    if (locationIp["country_code"] == ""):
        return row
    else:
        # row["country"] = locationIp["country_code"]
        if (row["currency"] is None):
            row["currency"] = currency_country[locationIp["country_code"]]
        if(row["currency"] == "EUR"):
            row["price_eur"] = row["price"]
        elif (row["currency"] == "BYR"):
            return row
        else:
            row["price_eur"] = row["price"] * euro_rate[row["currency"].lower()]["rate"]
    return row

In [207]:
# data["country"] = None
data = data.apply(lambda row: get_eur_price(row, currency_country, euro_rate), axis=1)

In [208]:
data

,username,ip_address,product,price,infos,currency,price_eur
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.80,May contain sugar,None,NaN
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,None,NaN
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,None,NaN
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,USD,390.949491
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JPY,115168.584952
...,...,...,...,...,...,...,...
195,ccannop5f,174.5.73.129,Nantucket Orange Juice,713.60,"Contains gluten, mustard, sugar, milk and fish",CAD,1045.116216
196,lprovis5g,191.69.45.257,"Jam - Strawberry, 20 Ml Jar",107.78,Contains gluten and sugar,None,NaN
197,iollarenshaw5h,206.30.25.226,"Juice - Apple, 341 Ml",447.17,May contain gluten and soja,RUB,31799.206492
198,mjablonski5i,44.133.211.182,"Mushroom - Chantrelle, Fresh",632.29,Ingredients: soja and sugar,USD,705.964455


In [210]:
data = data.dropna(subset=["price_eur"])

In [140]:
import string

data.infos = data.infos.str.lower()
data.infos = data.infos.str.replace(',', '')
data.infos = data.infos.str.replace(':', '')

# data.groupby("infos").count()

In [98]:
data.infos = data.infos.str.split()
dataProduct = data.explode("infos")["infos"]

In [142]:
dataProduct.head(10)
infosDummy = pd.get_dummies(dataProduct, columns=["infos"])
infosDummy

,and,contain,contains,egg,fish,gluten,ingredients,may,milk,mustard,peanut,soja,sugar
0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,0,1,0,0,0,0,0,0,0,0,0,0,0
199,0,0,0,0,0,1,0,0,0,0,0,0,0
199,0,0,0,0,0,0,0,0,0,0,0,0,1
199,1,0,0,0,0,0,0,0,0,0,0,0,0


In [143]:
infosDummy = infosDummy.groupby(level=0).sum()
infosDummy = infosDummy.drop(["may","and","contain","contains","ingredients"], axis=1)

In [102]:
infosDummy.head()

,egg,fish,gluten,milk,mustard,peanut,soja,sugar
0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,1,0,0
2,0,1,0,0,1,0,0,0
3,0,0,1,0,0,0,0,0
4,1,1,0,0,0,0,0,1


In [144]:
data.join(infosDummy)

,username,ip_address,product,price,infos,currency,egg,fish,gluten,milk,mustard,peanut,soja,sugar
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,contains gluten,USD,0,0,1,0,0,0,0,0
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,may contain sugar egg and fish,JPY,1,1,0,0,0,0,0,1
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33,ingredients sugar and milk,MGA,0,0,0,1,0,0,0,1
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,contains sugar,MXN,0,0,0,0,0,0,0,1
8,epridham8,187.129.113.105,Dried Figs,88.05,ingredients sugar milk and fish,MXN,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,ecasserley5b,82.78.162.116,Cup - 8oz Coffee Perforated,714.56,ingredients sugar and fish,RON,0,1,0,0,0,0,0,1
193,tliddyard5d,93.22.46.177,Shrimp - Black Tiger 6 - 8,300.03,ingredients sugar egg and fish,EUR,1,1,0,0,0,0,0,1
195,ccannop5f,174.5.73.129,Nantucket Orange Juice,713.6,contains gluten mustard sugar milk and fish,CAD,0,1,1,1,1,0,0,1
197,iollarenshaw5h,206.30.25.226,"Juice - Apple, 341 Ml",447.17,may contain gluten and soja,RUB,0,0,1,0,0,0,1,0


In [164]:
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 160 entries, 3 to 198
Data columns (total 6 columns):
username      160 non-null object
ip_address    160 non-null object
product       160 non-null object
price         160 non-null object
infos         160 non-null object
currency      160 non-null object
dtypes: object(6)
memory usage: 13.8+ KB


In [174]:
import numpy as np
data["price"].astype(np.float)

3      350.15
4      949.79
5      354.33
7      111.56
8       88.05
        ...  
191    714.56
193    300.03
195    713.60
197    447.17
198    632.29
Name: price, Length: 160, dtype: float64